# Requirements

In [25]:
%%capture
!pip install pdfplumber
import pdfplumber
import requests
import os
import pandas as pd


# Funciones

## Obtener odm provisorio
(Solo necesito las columnas DNI, Nombre, Apellido. Para que a partir del nombre habia hecho el join con ns_def.csv donde le asigno el sexo segun el nombre. Esto es asi porque luego en el orden definitivo, pusieron Apellido y Nombre junto, siendo dificil y quizas imposible separar los nombres compuestos y apellidos compuestos para poder obtener solo los nombres y asi estimar si es Femenino o Masculino)

In [26]:
def obtener_odm_provisorio(url_pdf: str, nombre_archivo: str = "odm_provisorio.pdf") -> pd.DataFrame:
    """
    Descarga, extrae y limpia datos de un PDF en un DataFrame listo para análisis.
    Args: url_pdf (str): URL del archivo PDF a descargar.
          nombre_archivo (str): Nombre con que se guardará el PDF localmente.
    Returns:  pd.DataFrame: DataFrame
    """
    # Descargar PDF si no existe localmente
    if not os.path.isfile(nombre_archivo):
        r = requests.get(url_pdf)
        if r.status_code == 200:
            with open(nombre_archivo, "wb") as f:
                f.write(r.content)
        else:
            print(f"Error downloading PDF from {url_pdf}. Status code: {r.status_code}")
            return None

    # Extraer tablas del PDF
    data = []
    try:
        with pdfplumber.open(nombre_archivo) as pdf:
            for page in pdf.pages:
                table = page.extract_table()
                if table:
                    data.extend(table)
    except Exception as e:
        print(f"Error opening or reading PDF file {nombre_archivo}: {e}")
        return None

    dfodmp = pd.DataFrame(data[1:], columns=data[1])
    dfodmp.columns = dfodmp.columns.str.replace("\n", " ", regex=True)
    rename_dict = {
        "Número de documento": "DNI",
        "Apellido": "APELLIDO",
        "Nombre": "NOMBRE" }
    dfodmp = dfodmp.rename(columns=rename_dict)
    dfodmp['DNI'] = dfodmp['DNI'].astype(str)

    return dfodmp

## Obtener ODM definitivo

In [27]:
def obtener_ODM2025(url_pdf: str, nombre_archivo: str = "ODM2025.pdf") -> pd.DataFrame:
    """
    Descarga, extrae y limpia datos de un PDF en un DataFrame listo para análisis.
    Args: url_pdf (str): URL del archivo PDF a descargar.
          nombre_archivo (str): Nombre con que se guardará el PDF localmente.
    Returns:  pd.DataFrame: DataFrame
    """
    # Descargar PDF si no existe localmente
    if not os.path.isfile(nombre_archivo):
        r = requests.get(url_pdf)
        if r.status_code == 200:
            with open(nombre_archivo, "wb") as f:
                f.write(r.content)
        else:
            print(f"Error downloading PDF from {url_pdf}. Status code: {r.status_code}")
            return None

    # Extraer tablas del PDF
    data = []
    try:
        with pdfplumber.open(nombre_archivo) as pdf:
            for page in pdf.pages:
                table = page.extract_table()
                if table:
                    data.extend(table)
    except Exception as e:
        print(f"Error opening or reading PDF file {nombre_archivo}: {e}")
        return None

    dfODM2025 = pd.DataFrame(data[1:], columns=data[1])
    dfODM2025.columns = dfODM2025.columns.str.replace("\n", " ", regex=True)
    columnas_a_eliminar = ['Apellido y Nombre']
    dfODM2025.drop(columns=columnas_a_eliminar, inplace=True)

    return dfODM2025

## Join odm (Nombre, Apellido) con ODM, a partir del DNI

In [28]:
def mergeODFS (df1, df2):
  # Supongamos que ya tienes los DataFrames:
  # df1 con columnas: DNI, NOMBRE, APELLIDO (dfodm)
  # df2 con columnas incluyendo DNI, NOMBRE, APELLIDO, PUNTAJE_CRUDO, ODM_CRUDO, PROMEDIO, ESPECIALIDAD (dfODM2025)

  # Hacer merge con df1 para traer NOMBRE y APELLIDO
  df = df2.merge(df1[['DNI', 'NOMBRE', 'APELLIDO']], on='DNI', how='left')

  return df

## Limpiar el df

In [29]:
def limpiar_df(df):

    # Renombrar columnas
    rename_dict = {
        "Institución formadora": "UNIVERSIDAD",
        "Promedio": "PROMEDIO_CARRERA",
        "Apellido": "APELLIDO",
        "Nombre": "NOMBRE",
        "Fecha de Expedición de Título": "FECHA_TITULO",
        "Especialidad": "ESPECIALIDAD",
        "Puntaje obtenido en el examen": "NOTA_EXAMEN",
        "Tipo Uni": "TIPO_UNI",
        "Componente": "COMPONENTE",
        "Puntaje Final": "PUNTAJE"
    }
    df = df.rename(columns=rename_dict)

    # Eliminar filas no deseadas
    df = df[~df.isin(["DNI"]).any(axis=1)] # filas donde se repite el encabezado
    df = df[df['DNI'].str.strip().astype(bool)]  # elimina filas vacías (con la celda de apellido vacio)
    df = df.dropna().reset_index(drop=True) # reindexar

    # Limpiar saltos de línea en nombres de universidades reemplazándolos por espacio
    df["UNIVERSIDAD"] = df["UNIVERSIDAD"].str.replace("\n", " ", regex=False)

    # Reemplazos globales en todas las celdas
    df = df.replace({r'\n': ' ', "En trámite": "30-06-2025"}, regex=True) #crear el espacio entre los nombres en vez de "\n" y poner la fecha que elegi 30 junio 2025 en vez de "en tramite"
    # Tiempo entre recibido y el examen (1 julio 2025)
    # Definir la fecha cero
    fecha_cero = pd.to_datetime("2025-07-01")

    # Ajustes de formato
    df["FECHA_TITULO"] = pd.to_datetime(df["FECHA_TITULO"], format="%d-%m-%Y", errors="coerce")

    # Floats
    cols_f = ["PROMEDIO_CARRERA", "PUNTAJE"]
    df[cols_f] = df[cols_f].replace(",", ".", regex=True).replace("", float('nan')).astype(float)
    # Enteros
    cols_i = ['NOTA_EXAMEN', 'COMPONENTE', 'ODM']
    for col in cols_i:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    df["COMPONENTE"] = df["COMPONENTE"].fillna(0)

    df['PUNTAJE_CRUDO'] = df['PUNTAJE']-df['COMPONENTE']



    # Asegurar que FECHA_TITULO es datetime
    df["FECHA_TITULO"] = pd.to_datetime(df["FECHA_TITULO"], format="%d-%m-%Y", errors="coerce")
    # Calcular diferencia en días
    df["DIAS_DESDE_TITULO"] = (fecha_cero - df["FECHA_TITULO"]).dt.days

    return df

## Mapeo de SEXO segun NOMBRE

In [30]:
def mapear_sexo_por_primer_nombre(df, url, nombre_col_original='NOMBRE', sexo_col='SEXO'):

    # Descargar y leer el archivo CSV si no existe localmente
    # Nombre del archivo local
    file_name = url.split("/")[-1]

    if not os.path.isfile(file_name):
        r = requests.get(url)
        with open(file_name, "wb") as f:
            f.write(r.content)

    ns_def = pd.read_csv(file_name)

    # Renombrar columna del archivo descargado para homogeneizar
    rename_dict = {"primer_nombre": "NOMBRE"}
    ns_def = ns_def.rename(columns=rename_dict)

    # Extraer primer nombre, limpiar y pasar a mayúscula
    df['primer_nombre'] = df[nombre_col_original].apply(lambda x: x.split()[0] if isinstance(x, str) else "")
    df['primer_nombre'] = df['primer_nombre'].str.strip().str.upper()
    ns_def['NOMBRE'] = ns_def['NOMBRE'].str.strip().str.upper()

    # Crear diccionario para mapeo de sexo
    dic_sexo = dict(zip(ns_def['NOMBRE'], ns_def[sexo_col]))

    # Mapear sexo usando el primer nombre
    df['SEXO'] = df['primer_nombre'].map(dic_sexo)

    # Marcar como 'ND' los casos sin coincidencia
    df['SEXO'] = df['SEXO'].fillna('ND')

    # Eliminar columna auxiliar
    df.drop(columns=['primer_nombre'], inplace=True)

    return df

## ORIGEN de postulante segun DNI >/<50millones

In [31]:
def asignar_origen(df, columna_dni='DNI'):
    # Crear columna ORIGEN según condición del DNI
    df['DNI'] = pd.to_numeric(df['DNI'], errors='coerce').astype('Int64')
    df['ORIGEN'] = df[columna_dni].apply(lambda x: 'arg' if x < 50000000 else 'extr')

    return df

## ODM alternativos:

In [32]:
# ODM_CRUDO: sin los 5 puntos mas a TIPO_UNI == N (o sea con puntaje crudo)
def asignar_ODM_crudo(df):
    df = df.sort_values(
        by=['ESPECIALIDAD', 'PUNTAJE_CRUDO', 'NOTA_EXAMEN', 'PROMEDIO_CARRERA', 'DNI'],
        ascending=[True, False, False, False, True]
    )
    df['ODM_CRUDO'] = df.groupby('ESPECIALIDAD').cumcount() + 1
    return df

# ODM_GLOBAL_CRUDO sin agrupar especialidades, con puntaje crudo
def ODM_global_crudo(df):
    df = df.sort_values(
        by=['PUNTAJE_CRUDO', 'NOTA_EXAMEN', 'PROMEDIO_CARRERA', 'DNI'],
        ascending=[False, False, False, True]).reset_index(drop=True)
    df['ODM_GLOBAL_CRUDO'] = df.index + 1
    return df

# ODM_GLOBAL con puntaje que se usi con 5 puntos nacionales, sin agrupar especialidades
def ODM_global(df):
    df = df.sort_values(
        by=['PUNTAJE', 'NOTA_EXAMEN', 'PROMEDIO_CARRERA', 'DNI'],
        ascending=[False, False, False, True]).reset_index(drop=True)
    df['ODM_GLOBAL'] = df.index + 1
    return df


## Mapeo de UNIVERSIDADES
(En un primer momento identifique en el odm provisorio que los nombres de UNIVERSIDADES no estaban normalizado y tambien que habia hosptales entre esos nombres: hospitales argentinos sin universidad de medicina. A estos los asigne a la UBA.)
(PENDIENTE: chequear en ODM definitivo si hay estos errores en las universidades)

In [33]:
def mapear_universidades(df, file_name, nombre_col_original='UNIVERSIDAD'):

    # Descargar y leer el archivo CSV si no existe localmente
    # Nombre del archivo local

    universidades = pd.read_csv(file_name)

    # Hacer merge con el df original usando la columna UNIVERSIDAD como clave
    df = df.merge(universidades[['UNIVERSIDAD','UNI', 'CLASE_UNI', 'PAIS_UNI','CIUDAD_UNI','lat','long']],
                         left_on = nombre_col_original, right_on = 'UNIVERSIDAD', how='left')

    # Eliminar la columna original de universidad
    #df = df.drop(columns=[nombre_col_original])

    return df

# Main

In [34]:
urlodm = "https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/main/generar_data/odm_provisorio.pdf"
dfodmp = obtener_odm_provisorio(urlodm)

urlODM2025 = "https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/main/generar_data/ODM2025.pdf"
dfODM2025 = obtener_ODM2025(urlODM2025)


df = mergeODFS(dfodmp, dfODM2025)
df = limpiar_df(df)

urlsexo = "https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/main/generar_data/ns_def.csv"
df = mapear_sexo_por_primer_nombre(df, urlsexo, nombre_col_original='NOMBRE', sexo_col='SEXO')

df = asignar_origen(df, columna_dni='DNI')

df = asignar_ODM_crudo(df)

urluni = "https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/main/generar_data/universidades.csv"
df = mapear_universidades(df, urluni, nombre_col_original='UNIVERSIDAD')

df = ODM_global(df)
df = ODM_global_crudo(df)

In [35]:
df

,DNI,UNIVERSIDAD,ESPECIALIDAD,PROMEDIO_CARRERA,FECHA_TITULO,NOTA_EXAMEN,TIPO_UNI,COMPONENTE,PUNTAJE,ODM,...,ORIGEN,ODM_CRUDO,UNI,CLASE_UNI,PAIS_UNI,CIUDAD_UNI,lat,long,ODM_GLOBAL,ODM_GLOBAL_CRUDO
0,42011937,UNIVERSIDAD DE BUENOS AIRES,Pediatría y pediátricas articuladas,9.07,2024-11-13,93,N,5,60.57,1,...,arg,1,UNIVERSIDAD DE BUENOS AIRES,Pública,Argentina,Buenos Aires,-34.6037,-58.3816,1,1
1,43418248,UNIVERSIDAD NACIONAL DE CUYO,Clínica médica,9.19,2025-06-30,92,N,5,60.19,1,...,arg,1,UNIVERSIDAD NACIONAL DE CUYO,Pública,Argentina,Mendoza,-32.8895,-68.8458,2,2
2,41280789,UNIVERSIDAD NACIONAL DE CORDOBA,Neurocirugía,9.07,2024-11-27,92,N,5,60.07,1,...,arg,1,UNIVERSIDAD NACIONAL DE CORDOBA,Pública,Argentina,Cordoba,-31.4201,-64.1888,3,3
3,40993904,UNIVERSIDAD DE BUENOS AIRES,Cirugía infantil (cirugía pediátrica),9.02,2024-05-10,92,N,5,60.02,1,...,arg,1,UNIVERSIDAD DE BUENOS AIRES,Pública,Argentina,Buenos Aires,-34.6037,-58.3816,4,4
4,42657933,UNIVERSIDAD DE BUENOS AIRES,Tocoginecología,8.91,2025-06-30,92,N,5,59.91,1,...,arg,1,UNIVERSIDAD DE BUENOS AIRES,Pública,Argentina,Buenos Aires,-34.6037,-58.3816,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6114,96244695,UNIVERSIDAD METROPOLITANA DE BARRANQUILLA,Oftalmología,0.00,2016-03-30,37,E,0,18.50,320,...,extr,321,UNIVERSIDAD METROPOLITANA DE BARRANQUILLA,Privada,Colombia,Barranquilla,10.9685,-74.7813,6115,6115
6115,95746273,UNIVERSIDAD TECNICA DE ORURO,Ortopedia y traumatología,0.00,2012-11-29,35,E,0,17.50,464,...,extr,464,UNIVERSIDAD TECNICA DE ORURO,Pública,Bolivia,Oruro,-17.9833,-67.1167,6116,6116
6116,96231679,UNIVERSIDAD METROPOLITANA DE BARRANQUILLA,Diagnóstico por imágenes,0.00,2019-07-26,33,E,0,16.50,370,...,extr,370,UNIVERSIDAD METROPOLITANA DE BARRANQUILLA,Privada,Colombia,Barranquilla,10.9685,-74.7813,6117,6117
6117,96426896,UNIVERSIDAD AUTONOMA DE BUCARAMANGA (UNAB),Psiquiatría,0.00,2022-12-20,33,E,0,16.50,272,...,extr,274,UNIVERSIDAD AUTONOMA DE BUCARAMANGA (UNAB),Privada,Colombia,Bucaramanga,7.1254,-73.1198,6118,6118


In [36]:
nuevo_orden = ['DNI', 'NOMBRE', 'APELLIDO', 'SEXO', 'ORIGEN',
               'UNI','TIPO_UNI', 'PAIS_UNI', 'CIUDAD_UNI', 'lat', 'long', 'CLASE_UNI',
               'FECHA_TITULO', 'DIAS_DESDE_TITULO', 'PROMEDIO_CARRERA', 'ESPECIALIDAD',
               'NOTA_EXAMEN', 'COMPONENTE', 'PUNTAJE', 'PUNTAJE_CRUDO', 'ODM', 'ODM_CRUDO',
               'ODM_GLOBAL','ODM_GLOBAL_CRUDO']

df = df[nuevo_orden]
df

,DNI,NOMBRE,APELLIDO,SEXO,ORIGEN,UNI,TIPO_UNI,PAIS_UNI,CIUDAD_UNI,lat,...,PROMEDIO_CARRERA,ESPECIALIDAD,NOTA_EXAMEN,COMPONENTE,PUNTAJE,PUNTAJE_CRUDO,ODM,ODM_CRUDO,ODM_GLOBAL,ODM_GLOBAL_CRUDO
0,42011937,NATALIA BELEN,BARROS QUINTEROS,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,9.07,Pediatría y pediátricas articuladas,93,5,60.57,55.57,1,1,1,1
1,43418248,EUGENIA LOURDES,REJON COCUZZA,F,arg,UNIVERSIDAD NACIONAL DE CUYO,N,Argentina,Mendoza,-32.8895,...,9.19,Clínica médica,92,5,60.19,55.19,1,1,2,2
2,41280789,MARIANO ANDRES,GATTI GOMEZ,M,arg,UNIVERSIDAD NACIONAL DE CORDOBA,N,Argentina,Cordoba,-31.4201,...,9.07,Neurocirugía,92,5,60.07,55.07,1,1,3,3
3,40993904,LAURA,DOLLER,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,9.02,Cirugía infantil (cirugía pediátrica),92,5,60.02,55.02,1,1,4,4
4,42657933,CLARA,ROLLERI CHAHER,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,8.91,Tocoginecología,92,5,59.91,54.91,1,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6114,96244695,ERNESTO FIDEL,CASTILLA POLO,M,extr,UNIVERSIDAD METROPOLITANA DE BARRANQUILLA,E,Colombia,Barranquilla,10.9685,...,0.00,Oftalmología,37,0,18.50,18.5,320,321,6115,6115
6115,95746273,RAMIRO ANGELO,GALLARDO ARAMAYO,M,extr,UNIVERSIDAD TECNICA DE ORURO,E,Bolivia,Oruro,-17.9833,...,0.00,Ortopedia y traumatología,35,0,17.50,17.5,464,464,6116,6116
6116,96231679,WILLIAM FERNANDO,RAMOS MOSQUERA,M,extr,UNIVERSIDAD METROPOLITANA DE BARRANQUILLA,E,Colombia,Barranquilla,10.9685,...,0.00,Diagnóstico por imágenes,33,0,16.50,16.5,370,370,6117,6117
6117,96426896,MARIA PAULA,VALLEJO TAPIAS,F,extr,UNIVERSIDAD AUTONOMA DE BUCARAMANGA (UNAB),E,Colombia,Bucaramanga,7.1254,...,0.00,Psiquiatría,33,0,16.50,16.5,272,274,6118,6118


# Descarga BaseDatos

In [37]:
df.to_csv('Base_ODM2025.csv', index=False)
from google.colab import files
files.download('Base_ODM2025.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Analisis

In [ ]:
df.info()

In [ ]:
df.describe(include=object)

In [ ]:
df['ESPECIALIDAD'].value_counts()

In [ ]:
df[df['PAIS_UNI']=='Argentina']['UNI'].value_counts()

In [ ]:
df[['PROMEDIO_CARRERA', 'NOTA_EXAMEN', 'PUNTAJE','PUNTAJE_CRUDO']].describe()